<a href="https://colab.research.google.com/github/ayushmakafle/SalesForecasting/blob/main/salesforecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

%pip install -Uq upgini catboost

In [ ]:
from os.path import exists 
import pandas as pd 

df_path = 'train.csv.zip'  if exists('train.csv.zip') else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path) #reading csv file and putting it into a dataframe called df
df = df.sample (n=19_000 , random_state=0)
df['store'] = df['store'].astype(str)
df['item'] = df['item'].astype(str)
df['date']= pd.to_datetime(df['date'])

df.sort_values('date', inplace=True)
df.reset_index(inplace=True, drop=True)
df.head() #gives us the top five rows

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [ ]:
train = df[df['date']<'2017-01-01']
test = df[df['date']>='2017-01-01']

In [ ]:
train_features = train.drop(columns=['sales'])
train_target = train['sales']
test_features = test.drop(columns=['sales'])
test_target=test['sales']


Enrich Features

In [ ]:
from upgini import FeaturesEnricher , SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys={
        'date' : SearchKey.DATE,
    },
    cv= CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features,test_target)])

<IPython.core.display.Javascript object>

Detected task type: ModelTaskType.REGRESSION


Column name,Status,Description
target,All valid,All values in this column are good to go
date,All valid,All values in this column are good to go


Running search request with search_id=b0815520-94a6-4043-8bcb-a1893854004f
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done


28 relevant feature(s) found with the search keys: ['date'].


,feature_name,shap_value,coverage %,type
0,item,0.487726,100.0,CHARACTER
1,store,0.172106,100.0,CHARACTER
2,f_weather_pca_0_94efd18d,0.056047,100.0,NUMERIC
3,f_week_sin1_a71d22f6,0.044632,100.0,NUMERIC
4,f_week_cos1_d3d56d7f,0.029552,100.0,NUMERIC
5,f_weather_umap_48_66a91289,0.025132,100.0,NUMERIC
6,f_weather_umap_24_409427e4,0.019315,100.0,NUMERIC
7,f_weather_umap_33_b9760f68,0.014638,100.0,NUMERIC
8,f_year_cos1_cd165f8c,0.012112,100.0,NUMERIC
9,f_dow_jones_89547e1d,0.007461,100.0,NUMERIC


In [ ]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor (verbose=False, allow_writing_files= False, random_state=0 )

enricher.calculate_metrics(
    train_features,train_target,
    eval_set=([(test_features,test_target)]),
    estimator=model,
    scoring='mean_absolute_percentage_error' #defining how we want to score how accurate this model is

)

Calculating metrics...
Done


,match_rate,baseline mean_absolute_percentage_error,enriched mean_absolute_percentage_error,uplift
,,,,
train,100.0,0.255844,0.16662,0.089224
eval 1,100.0,0.243877,0.13113,0.112746


In [ ]:
enriched_train_features = enricher.transform(train_features, keep_input=True) #input=true so that existing columns remain there
enriched_test_features = enricher.transform(test_features,keep_input = True)
enriched_train_features.head()

90.39637% of the rows are fully duplicated


Column name,Status,Description
date,All valid,All values in this column are good to go


Running search request with search_id=73d454a6-c63e-4715-b89f-3f277911ffae
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Done
90.36176% of the rows are fully duplicated


Column name,Status,Description
date,All valid,All values in this column are good to go


Running search request with search_id=bd148bc2-351b-4eec-ad07-6fb27c2174c3
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Done


,date,f_cbpol_pca_3_2e94b9bf,f_cbpol_umap_1_34dc2149,f_cbpol_umap_6_f175da9a,f_cpi_pca_5_db7798a3,f_dow_jones_7d_to_7d_1y_shift_9628c89b,f_dow_jones_89547e1d,f_finance_umap_3_424d51ca,f_italy_game_cnt_9cfcfe65,f_mlending_approve_score_d4c33397,...,f_weather_umap_34_39fc3e94,f_weather_umap_35_436c04a5,f_weather_umap_43_4e9820c4,f_weather_umap_45_b348f420,f_weather_umap_48_66a91289,f_week_cos1_d3d56d7f,f_week_sin1_a71d22f6,f_year_cos1_cd165f8c,item,store
0,2013-01-01,-0.323471,4.815701,1.367325,-8.943169,1.065267,13104.139648,7.647812,0,0.338412,...,5.664261,4.76773,5.079482,4.923654,4.540985,0.62349,0.781831,0.98522,5,7
1,2013-01-01,-0.323471,4.815701,1.367325,-8.943169,1.065267,13104.139648,7.647812,0,0.338412,...,5.664261,4.76773,5.079482,4.923654,4.540985,0.62349,0.781831,0.98522,9,4
2,2013-01-01,-0.323471,4.815701,1.367325,-8.943169,1.065267,13104.139648,7.647812,0,0.338412,...,5.664261,4.76773,5.079482,4.923654,4.540985,0.62349,0.781831,0.98522,33,1
3,2013-01-01,-0.323471,4.815701,1.367325,-8.943169,1.065267,13104.139648,7.647812,0,0.338412,...,5.664261,4.76773,5.079482,4.923654,4.540985,0.62349,0.781831,0.98522,41,3
4,2013-01-01,-0.323471,4.815701,1.367325,-8.943169,1.065267,13104.139648,7.647812,0,0.338412,...,5.664261,4.76773,5.079482,4.923654,4.540985,0.62349,0.781831,0.98522,24,5


Train and evalutate model on original data set and enriched data set and compare the error and see which one turns out better 

In [ ]:
model.fit(train_features,train_target)
preds=model.predict(test_features)
eval_metric(test_target.values,preds,'SMAPE') #SMAPE:symmetric mean absolute percentage error , gives us percentage errors between what our target is actually supposed to be and what our model predicted


[37.65141857448004]

In [ ]:
model.fit(enriched_train_features, train_target)
preds=model.predict(enriched_test_features)
eval_metric(test_target.values , preds , 'SMAPE' )

[14.504497540797917]